In [1]:
import pyshark

In [4]:
#Something about needing an xcode install. Waiting until we are done with class so I don't screw it up

In [2]:
# Sniff from interface in real time
capture = pyshark.LiveCapture(interface='en0')
capture.sniff(timeout=20)

RuntimeError: This event loop is already running

In [3]:
cap = pyshark.FileCapture('http.cap', keep_packets=False)
def print_info_layer(packet):
  print("[Destination IP:]"+packet.ip.dst)
cap.apply_on_packets(print_info_layer)

FileNotFoundError: [Errno 2] No such file or directory: http.cap